In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import * 

import scipy.stats as spst
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

from itertools import product
import pickle

## 최종 테스트 파일 불러오기

In [78]:
final_data = pd.read_csv('final_data.csv')
final_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),...,부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간,holiday_1,Korean_holiday_1,year,month,time,weekend_1
0,2022-09-07 00:00:00,20.7,0.0,3.4,4.0,83.0,20.2,17.7,1003.1,1013.0,0.0,0.00,0.0,2000.0,17.7,...,16.5,1004.3,1012.3,0.0,0.00,0.0,5000.0,22.0,312.0,1,1,2022,9,0,0
1,2022-09-07 01:00:00,20.7,0.0,2.2,4.0,86.0,20.9,18.2,1003.3,1013.2,0.0,0.00,0.0,2000.0,18.1,...,16.7,1004.5,1012.6,0.0,0.00,0.0,5000.0,21.7,0.0,1,1,2022,9,1,0
2,2022-09-07 02:00:00,20.7,0.0,2.5,3.0,89.0,21.7,18.8,1003.3,1013.2,0.0,0.00,0.0,1871.0,18.2,...,16.1,1004.6,1012.7,0.0,0.00,0.0,5000.0,21.2,0.0,1,1,2022,9,2,0
3,2022-09-07 03:00:00,20.5,0.0,1.9,3.0,91.0,21.8,18.9,1003.8,1013.8,0.0,0.00,0.0,1155.0,17.5,...,15.2,1005.2,1013.3,0.0,0.00,0.0,5000.0,20.9,0.0,1,1,2022,9,3,0
4,2022-09-07 04:00:00,19.9,0.0,0.7,5.0,92.0,21.3,18.5,1004.0,1014.0,0.0,0.00,0.0,881.0,16.6,...,15.0,1005.3,1013.4,0.0,0.00,0.0,5000.0,20.5,0.0,1,1,2022,9,4,0
5,2022-09-07 05:00:00,19.1,0.0,1.3,3.0,92.0,20.2,17.7,1004.6,1014.5,0.0,0.00,0.0,711.0,15.7,...,14.0,1005.6,1013.7,0.0,0.00,0.0,5000.0,20.2,0.0,1,1,2022,9,5,0
6,2022-09-07 06:00:00,18.6,0.0,1.5,6.0,94.0,20.1,17.6,1005.3,1015.3,0.0,0.00,0.0,887.0,15.6,...,14.3,1006.4,1014.5,0.0,0.00,0.0,5000.0,20.0,0.0,1,1,2022,9,6,0
7,2022-09-07 07:00:00,18.0,0.0,0.7,1.0,93.0,19.1,16.8,1006.2,1016.2,1.0,0.12,0.0,1992.0,15.6,...,12.3,1007.2,1015.3,0.0,0.08,0.0,5000.0,20.4,0.0,1,1,2022,9,7,0
8,2022-09-07 08:00:00,19.3,0.0,1.2,0.0,89.0,19.9,17.4,1006.8,1016.8,2.0,0.74,0.0,2000.0,21.0,...,11.6,1007.6,1015.6,1.0,0.84,0.0,5000.0,22.6,0.0,1,1,2022,9,8,0
9,2022-09-07 09:00:00,21.6,0.0,2.0,0.0,81.0,20.8,18.1,1007.2,1017.1,2.0,1.53,0.0,2000.0,27.6,...,11.8,1007.9,1015.9,2.0,1.63,0.0,5000.0,25.6,0.0,1,1,2022,9,9,0


- 필요없는 변수 drop

In [79]:
final_data.drop(columns=['일시', '부산 적설(cm)'], axis=1, inplace=True)

- feature 이름 바꾸기

In [80]:
final_data.rename(columns={'weekend':'weekend_1', 'holiday':'holiday_1', 'Korean_holiday':'Korean_holiday_1'}, inplace=True)

In [81]:
final_data['weekend_1'] = 0

- 저장하기

In [82]:
final_data.to_csv('final_data.csv', index=False)

In [83]:
final_data

,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),...,부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 시정(10m),부산 지면온도(°C),소요시간,holiday_1,Korean_holiday_1,year,month,time,weekend_1
0,20.7,0.0,3.4,4.0,83.0,20.2,17.7,1003.1,1013.0,0.0,0.00,0.0,2000.0,17.7,21.4,...,18.8,16.5,1004.3,1012.3,0.0,0.00,5000.0,22.0,312.0,1,1,2022,9,0,0
1,20.7,0.0,2.2,4.0,86.0,20.9,18.2,1003.3,1013.2,0.0,0.00,0.0,2000.0,18.1,20.9,...,19.0,16.7,1004.5,1012.6,0.0,0.00,5000.0,21.7,0.0,1,1,2022,9,1,0
2,20.7,0.0,2.5,3.0,89.0,21.7,18.8,1003.3,1013.2,0.0,0.00,0.0,1871.0,18.2,20.5,...,18.3,16.1,1004.6,1012.7,0.0,0.00,5000.0,21.2,0.0,1,1,2022,9,2,0
3,20.5,0.0,1.9,3.0,91.0,21.8,18.9,1003.8,1013.8,0.0,0.00,0.0,1155.0,17.5,20.3,...,17.3,15.2,1005.2,1013.3,0.0,0.00,5000.0,20.9,0.0,1,1,2022,9,3,0
4,19.9,0.0,0.7,5.0,92.0,21.3,18.5,1004.0,1014.0,0.0,0.00,0.0,881.0,16.6,20.0,...,17.0,15.0,1005.3,1013.4,0.0,0.00,5000.0,20.5,0.0,1,1,2022,9,4,0
5,19.1,0.0,1.3,3.0,92.0,20.2,17.7,1004.6,1014.5,0.0,0.00,0.0,711.0,15.7,19.5,...,16.0,14.0,1005.6,1013.7,0.0,0.00,5000.0,20.2,0.0,1,1,2022,9,5,0
6,18.6,0.0,1.5,6.0,94.0,20.1,17.6,1005.3,1015.3,0.0,0.00,0.0,887.0,15.6,18.9,...,16.3,14.3,1006.4,1014.5,0.0,0.00,5000.0,20.0,0.0,1,1,2022,9,6,0
7,18.0,0.0,0.7,1.0,93.0,19.1,16.8,1006.2,1016.2,1.0,0.12,0.0,1992.0,15.6,19.6,...,14.3,12.3,1007.2,1015.3,0.0,0.08,5000.0,20.4,0.0,1,1,2022,9,7,0
8,19.3,0.0,1.2,0.0,89.0,19.9,17.4,1006.8,1016.8,2.0,0.74,0.0,2000.0,21.0,21.7,...,13.7,11.6,1007.6,1015.6,1.0,0.84,5000.0,22.6,0.0,1,1,2022,9,8,0
9,21.6,0.0,2.0,0.0,81.0,20.8,18.1,1007.2,1017.1,2.0,1.53,0.0,2000.0,27.6,23.9,...,13.8,11.8,1007.9,1015.9,2.0,1.63,5000.0,25.6,0.0,1,1,2022,9,9,0


## 최종 검증

- XGB, LGBM, SARIMAX 모델 사용

In [84]:
with open("model_xgb.pkl", "rb") as file:
        model_xgb_gs = pickle.load(file)
        
        
with open("model_lgbm.pkl", "rb") as file:
        model_lgbm_gs = pickle.load(file)
        
        
with open("model_SARIMAX.pkl", "rb") as file:
        model_SARIMAX = pickle.load(file)

### 최종 : SARIMAX 모델로 선택하여 검증 진행

In [97]:
pred = 312
for index, row in final_data.iterrows():
    row = final_data.iloc[[index]]
    row['소요시간'] = pred
    print(row['소요시간'])
    row_tmp = row.loc[:, ['서울 기온(°C)', '서울 강수량(mm)', '서울 풍속(m/s)', '서울 습도(%)',
                              '부산 기온(°C)', '부산 강수량(mm)', '부산 풍속(m/s)', '부산 습도(%)', 
                              '소요시간', 'holiday_1', 'weekend_1']]
    display(row)

    pred_SARIMAX = model_SARIMAX.forecast(1, exog=row_tmp)
    pred_SARIMAX = pred_SARIMAX+(271-pred_SARIMAX)*1.5
    
    pred = list(pred_SARIMAX)
    pred = int(np.round(pred))
    print(pred)
    

0    312
Name: 소요시간, dtype: int64


,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),...,부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 시정(10m),부산 지면온도(°C),소요시간,holiday_1,Korean_holiday_1,year,month,time,weekend_1
0,20.7,0.0,3.4,4.0,83.0,20.2,17.7,1003.1,1013.0,0.0,0.0,0.0,2000.0,17.7,21.4,...,18.8,16.5,1004.3,1012.3,0.0,0.0,5000.0,22.0,312,1,1,2022,9,0,0


297
1    297
Name: 소요시간, dtype: int64


,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),...,부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 시정(10m),부산 지면온도(°C),소요시간,holiday_1,Korean_holiday_1,year,month,time,weekend_1
1,20.7,0.0,2.2,4.0,86.0,20.9,18.2,1003.3,1013.2,0.0,0.0,0.0,2000.0,18.1,20.9,...,19.0,16.7,1004.5,1012.6,0.0,0.0,5000.0,21.7,297,1,1,2022,9,1,0


290
2    290
Name: 소요시간, dtype: int64


...

284
11    284
Name: 소요시간, dtype: int64


,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),...,부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 시정(10m),부산 지면온도(°C),소요시간,holiday_1,Korean_holiday_1,year,month,time,weekend_1
11,24.5,0.0,2.2,5.0,48.0,14.7,12.7,1007.6,1017.4,2.0,2.72,0.0,2000.0,34.6,27.0,...,18.4,16.2,1007.7,1015.6,2.0,2.87,5000.0,34.1,284,1,1,2022,9,11,0


282
12    282
Name: 소요시간, dtype: int64


,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),...,부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 시정(10m),부산 지면온도(°C),소요시간,holiday_1,Korean_holiday_1,year,month,time,weekend_1
12,25.9,0.0,2.5,3.0,36.0,11.9,9.6,1007.4,1017.2,2.0,3.15,0.0,2000.0,37.1,27.7,...,16.3,14.3,1007.3,1015.2,2.0,3.21,5000.0,36.7,282,1,1,2022,9,12,0


281


### 최종 결론

2022년 9월 9일 12시 교통 소요시간 = 281분 = 4시간 41분